In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import numpy as np

import xarray as xr
import xrft
import pandas as pd
import itertools


from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
client = Client(dashboard_address = '137.129.155.67:8789')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8789/status,
Dashboard: http://127.0.0.1:8789/status,Workers: 4
Total threads: 12,Total memory: 78.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36689,Workers: 4
Dashboard: http://127.0.0.1:8789/status,Total threads: 12
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://127.0.0.1:43307,Total threads: 3
Dashboard: http://127.0.0.1:36099/status,Memory: 19.65 GiB
Nanny: tcp://127.0.0.1:35627,


In [2]:
############################################################################
indir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/'
outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/SMOTHED_CLIM/'
minYear = '1990'
maxYear = '2020'
time_R = pd.date_range("1960-01-01", freq="3H", periods=366 * int(24/3))
#number of harmonics to keep
nbSampl = 4
nbHarm = 3  # 0 for the mean, 1 for the the annual cycle, etc...
nbHarmKeep = nbSampl*nbHarm
#Variable to smooth
var = ['tcwv']
############################################################################

for v in var :
	exp = 'clim_tcwv_smooth_ERA5_3H_1990_2020_4Dto3D'
	ds = xr.open_dataset(indir+exp+'.nc', chunks = {'latitude' : 10,'longitude' : 1}
		                         )

	# ## Selection
	# Dans la cellule au-dessus on a sortie dans un dataset la climatologie brute de ERA5, et on souhaite "smooth" le signal. On ne garde que les premières harmonique du signal. On utilise le package pour faire une transformée de Fourier sur des tableaux Xarray. https://xrft.readthedocs.io/en/latest/index.html

	tcwvhat = xrft.fft(ds['tcwv'], dim="time", true_phase=False, true_amplitude=True)
	tcwvhat = xr.where(tcwvhat.freq_time < -nbHarmKeep/(86400*366),  0., tcwvhat )
	tcwvhat = xr.where(tcwvhat.freq_time > nbHarmKeep/(86400*366),  0., tcwvhat)
	tcwv_Sm = xrft.ifft(tcwvhat, dim = 'freq_time', true_phase=False, true_amplitude=True) # Signal in direct space
	tcwv_Sm['time'] = ds.time
# 	
	ds = tcwv_Sm.real.to_dataset(name = v+'_smooth')
# 	ds = ds.compute()
# 	ds.to_netcdf(outdir + 'clim_'+v+'_smooth_ERA5_3H_'+str(minYear)+'_'+str(maxYear)+'.nc')

# print('script end')

/home/durandy/miniconda3/envs/croco/lib/python3.7/site-packages/xrft/xrft.py:556: FutureWarning: Default idft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [3]:
# da = ds.chunk(chunks = {'time' : 1})
# da

In [4]:
def split_by_chunks(dataset):
    chunk_slices = {}
    for dim, chunks in dataset.chunks.items():
        slices = []
        start = 0
        for chunk in chunks:
            if start >= dataset.sizes[dim]:
                break
            stop = start + chunk
            slices.append(slice(start, stop))
            start = stop
        chunk_slices[dim] = slices
    for slices in itertools.product(*chunk_slices.values()):
        selection = dict(zip(chunk_slices.keys(), slices))
        yield dataset[selection]
        
datasets = list(split_by_chunks(ds))
# print(datasets)

def create_filepath(ds, prefix='filename', root_path="/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/temp/test/"):
    """
    Generate a filepath when given an xarray dataset
    """
    start = int(ds.latitude.data[0])
    start_L = int(ds.latitude.data[-1])
    end = (ds.longitude.data[-1])
    filepath = f'{root_path}/{prefix}_{start}_{end}_{start_L}.nc'
    return filepath

paths = [create_filepath(ds) for ds in datasets]

In [ ]:
xr.save_mfdataset(datasets=datasets, paths=paths)
# paths

In [ ]:
paths

In [1]:
32/4

8.0